# overview
Recomendation system menggunakan cosine similarity

types of recomedation system

- content based
content berbasis similarity, similarity based of content (tags)
->sportify

- colaborative filtering based
user interest about behavior user, similarity by behavior user
->facebook

-  hybrid 
gabungan antara content used by tags terhadap collaborative filtering by behavior data

in the project its used by content based system for recomendation system, because the collaborative must have user to behavior data by used the content.

# project overflow

Data -> preprocessing -> model building -> connect api (website) ->develope 

In [1]:
# Data
"""
-> TMDB 5000 Movie Data set by Kaggle data set 
"""
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname,filename))


In [2]:
#load data
import numpy as np # linear algebra
import pandas as pd # data processing
movies = pd.read_csv("Datasets/tmdb_5000_movies.csv")
credits= pd.read_csv("Datasets/tmdb_5000_credits.csv")



In [3]:
# examine data
movies.head(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
credits.head(5)
# credits.head(1)["crew"].values

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [5]:
# analysis quantitu data
print(movies.shape)
print(credits.shape)

(4803, 20)
(4803, 4)


In [6]:
# less hectice data frame, most merge data of 2 data frame on one used merge
movies= pd.merge(movies,credits,on="title")

In [7]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [8]:
# check corelation field
corr_matriks = movies.corr()
corr_matriks["id"]

budget         -0.089099
id              1.000000
popularity      0.031240
revenue        -0.050530
runtime        -0.153580
vote_average   -0.270692
vote_count     -0.004096
movie_id        0.999843
Name: id, dtype: float64

In [9]:
# cut :

# budget ->not relevant for the data
# homepage -> not relevant for quality the similarity data
# id -> not relevant by id for data
# original_language -> not balanced data all because have the eng
# original_title ->original title cut of because the similarity have to title
# popularity ->if use this good but give the field ill complex
# production_comapny -> not relevant for quality
# production_countries -> this too
# release-date(not sure) -> i think is used but not sure becauase the date


# use the important field 

movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [10]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4809 non-null   int64 
 1   title     4809 non-null   object
 2   overview  4806 non-null   object
 3   genres    4809 non-null   object
 4   keywords  4809 non-null   object
 5   cast      4809 non-null   object
 6   crew      4809 non-null   object
dtypes: int64(1), object(6)
memory usage: 300.6+ KB


In [11]:
# check data missing -> haved the null value
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [12]:
# overview haved the null value just replace dropna,the dropna default axis=0->row
movies.dropna(inplace=True)
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [13]:
# cek redudansi data
movies.duplicated().sum()

0

In [14]:
# overview,genres,keywords,cast,crew have the dic list data, need convert that
# like this
print(movies.iloc[0].genres)
import ast # abstract syntax tree convert to the abstract tree 

[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]


In [15]:
# the result is dictionary value, convert by ast and take per value by dic
# make convert function
def convert(text):
    L=[]
    for i in ast.literal_eval(text):
        L.append(i["name"])
    return L


In [16]:
movies["genres"]=movies["genres"].apply(convert)
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [17]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [18]:
movies["keywords"]=movies["keywords"].apply(convert)

In [19]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [20]:
movies["cast"]= movies["cast"].apply(convert)

In [21]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman, A...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [22]:
movies["crew"]=movies["crew"].apply(convert)
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Stephen E. Rivkin, Rick Carter, Christopher B..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[Dariusz Wolski, Gore Verbinski, Jerry Bruckhe..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[Thomas Newman, Sam Mendes, Anna Pinnock, John..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman, A...","[Hans Zimmer, Charles Roven, Christopher Nolan..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[Andrew Stanton, Andrew Stanton, John Lasseter..."


done the data dic value on the filed genres, keywords, cast and crew must be clear, use the abstract tree mthode to convert on the value from dict value typed

In [23]:
# this function to use ast for litera_eval->safelty evaluate expresion nod eor string
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [24]:
# if want to more complex to cast field
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 

In [25]:
# if want to want more complex to direcotr field
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 

In [26]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
188,27576,Salt,"As a CIA officer, Evelyn Salt swore an oath to...","[Action, Mystery, Thriller]","[assassination, spy, cia, kidnapping, cold war...","[Angelina Jolie, Liev Schreiber, Chiwetel Ejio...","[Avy Kaufman, James Newton Howard, Stuart Bair..."
2140,239678,This Is Where I Leave You,"When their father passes away, four grown, wor...","[Drama, Comedy]","[based on novel, funeral, dysfunctional family...","[Jason Bateman, Tina Fey, Adam Driver, Rose By...","[Michael Giacchino, Paula Weinstein, Shawn Lev..."
2310,22479,Kansas City,"Robert Altman's story is a riff on race, class...","[Thriller, Drama, Music, Crime]","[politician, kansas city, election, jazz music]","[Jennifer Jason Leigh, Miranda Richardson, Har...","[Geraldine Peroni, Elisabeth Leustig, Richard ..."
2267,10448,Rapa Nui,Inter-tribal rivalry leads to a competition to...,[Adventure],"[in love with enemy, indigenous, island]","[Jason Scott Lee, Esai Morales, Sandrine Holt,...","[Barrie M. Osborne, Kevin Costner, Stewart Cop..."
4526,35199,Wordplay,From the masters who create the mind-bending d...,[Documentary],"[competition, documentary, contest, crossword ...","[Will Shortz, Merl Reagle, Tyler Hinman, Trip ...","[Patrick Creadon, Patrick Creadon, Christine O..."


In [27]:
# on the overview look at that, so long string so hard to ansume for manchine
## must to split that
movies["overview"]=movies["overview"].apply(lambda x:x.split())

In [28]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
1275,4965,Impostor,"[Gary, Sinise, plays, Spencer, Olham,, a, top-...","[Action, Science Fiction, Thriller]","[android, alien]","[Gary Sinise, Madeleine Stowe, Mekhi Phifer, V...","[Gary Sinise, Heidi Levitt, John Papsidera, Ph..."
4368,10183,Frozen River,"[Ray, Eddy,, an, upstate, New, York, trailer, ...",[Drama],"[human trafficking, woman director]","[Melissa Leo, Misty Upham, Charlie McDermott, ...","[Kate Williams, Courtney Hunt, Courtney Hunt, ..."
4107,331190,Captive,"[Based, on, a, miraculous, true, story, that, ...","[Crime, Drama, Thriller]","[hostage, based on true story, murder, indepen...","[Kate Mara, David Oyelowo, Michael Kenneth Wil...","[Ralph Winter, Melissa Kent, David Oyelowo, Je..."
4677,10218,Swingers,"[This, is, a, story, about, Mike,, a, guy, who...","[Comedy, Romance]","[independent film, stuffed animal, hollywood, ...","[Jon Favreau, Vince Vaughn, Ron Livingston, Pa...","[Stephen Mirrione, Doug Liman, Doug Liman, Jon..."
1498,8055,The Reader,"[The, story, of, Michael, Berg,, a, German, la...","[Drama, Romance]","[germany, war crimes, trial, female prisoner, ...","[Kate Winslet, Ralph Fiennes, David Kross, Jea...","[Roger Deakins, Stephen Daldry, Jina Jay, Sydn..."


In [29]:
"""
like that the spasi on the string is hard to ansume is problem be ansume different

babi guling
babiguling

on the model manchine is means different, must use the some funtion to convert that
"""
def collapse(L):
    L1=[]
    for i in L:
        L1.append(i.replace(" ",""))# change the spasi on the one word
    return L1


In [30]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [31]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...","[StephenE.Rivkin, RickCarter, ChristopherBoyes..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...","[DariuszWolski, GoreVerbinski, JerryBruckheime..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...","[ThomasNewman, SamMendes, AnnaPinnock, JohnLog..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman, Anne...","[HansZimmer, CharlesRoven, ChristopherNolan, C..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...","[AndrewStanton, AndrewStanton, JohnLasseter, C..."


In [32]:
# create tags for content based similairty point
movies["tags"] =  movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [33]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...","[StephenE.Rivkin, RickCarter, ChristopherBoyes...","[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...","[DariuszWolski, GoreVerbinski, JerryBruckheime...","[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...","[ThomasNewman, SamMendes, AnnaPinnock, JohnLog...","[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman, Anne...","[HansZimmer, CharlesRoven, ChristopherNolan, C...","[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...","[AndrewStanton, AndrewStanton, JohnLasseter, C...","[John, Carter, is, a, war-weary,, former, mili..."


In [34]:
new_movies = movies.drop(columns=['overview','genres','keywords','cast','crew'])
new_movies.head()

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [35]:
# give back the data seperti sbeelumnya on specifik on tags
# dikarenakan supaya memudahkan dalam proses create tags
new_movies['tags'] = new_movies['tags'].apply(lambda x: " ".join(x))
new_movies.head()
# and the data be means speare per field, setiap koma diartika setiap field yang
# berkaitan pada tags (overview','genres','keywords','cast','crew)
# sebelumnya diartikan per kata skrng per field dalam tags oleh manchine
# 5 base point of tags will got


,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [36]:
# look that
new_movies.iloc[0].tags

"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver StephenLang MichelleRodriguez GiovanniRibisi JoelDavidMoore CCHPounder WesStudi LazAlonso DileepRao MattGerald SeanAnthonyMoran JasonWhyte ScottLawrence KellyKilgour JamesPatrickPitt SeanPatrickMurphy PeterDillon KevinDorman KelsonHenderson DavidVanHorn JacobTomuri MichaelBlain-Rozgay JonCurry LukeHawker WoodySchultz PeterMensah SoniaYee JahnelCurfman IlramChoi KylaWarren LisaRoumain DebraWilson ChrisMala TaylorKibby JodieLandau JulieLamm CullenB.Madden JosephBradyMadden FrankieTorres AustinWilson SaraWilson TamicaWashington-Miller LucyBriant NathanM

In [37]:
# get all lower for easy like some on saw for model
new_movies["tags"]=new_movies["tags"].apply(lambda x:x.lower())

In [38]:
new_movies.iloc[0].tags

"in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver stephenlang michellerodriguez giovanniribisi joeldavidmoore cchpounder wesstudi lazalonso dileeprao mattgerald seananthonymoran jasonwhyte scottlawrence kellykilgour jamespatrickpitt seanpatrickmurphy peterdillon kevindorman kelsonhenderson davidvanhorn jacobtomuri michaelblain-rozgay joncurry lukehawker woodyschultz petermensah soniayee jahnelcurfman ilramchoi kylawarren lisaroumain debrawilson chrismala taylorkibby jodielandau julielamm cullenb.madden josephbradymadden frankietorres austinwilson sarawilson tamicawashington-miller lucybriant nathanm

In [39]:
# conver text to vector
"""
text -> vector
     -> text vector
used :bag of vectors, idfword

karena data pada tags sangat banyak di dalamnya berisikan field
maka dari itu dipelrukan sebuah perubahan data agar memeiliki arah dalam setiap cordinat
2d:
|
|
|
|-----(5,6,7,8)
|         :
------------------------->

    w1|w2|w3|......|w ndatamovie|
m1  3   5  6         1
m2
m3

(ndatamovie,ndatamovie) convert to -> (ndata,w1+w2)

convert every movie is vector
example max feature is 5000 for take 5000 sample movie
"""
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words="english")#same like steamportter
vector=cv.fit_transform(new_movies["tags"]).toarray()

In [40]:
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [41]:
print(vector.shape)
print(new_movies["tags"].shape)
# datanya yg dari 1 di pecah ke dalam beberapa field sehingga memudahkan dalam
# merepresentasikan data

(4806, 5000)
(4806,)


we have 4806 vector and 5000 field from before is one
..............5000
[   ]
:
:
:
4806

|           o

|       /

|   o

|
_________________________________________________
/-> similarity for setiap content row
untuk mencari / => gunakan cosine similarity kemiripan dengan mencari arah
di setiap dot vector gradient=>gradient based learning is used from cosine similarity

In [42]:
# cek cv 
cv.get_feature_names()

c:\Users\User\env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['000',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '1930s',
 '1940s',
 '1950s',
 '1960s',
 '1970s',
 '1980s',
 '19th',
 '19thcentury',
 '20',
 '20th',
 '24',
 '30',
 '3d',
 '40',
 '50',
 'aaroneckhart',
 'aaronseltzer',
 'aaronwilliams',
 'aaronzigman',
 'abandoned',
 'abigailbreslin',
 'ability',
 'able',
 'abrams',
 'abuse',
 'academy',
 'accepts',
 'accident',
 'accidentally',
 'account',
 'accused',
 'ackerman',
 'act',
 'action',
 'actions',
 'activities',
 'actor',
 'actress',
 'actually',
 'adam',
 'adamdale',
 'adamgoldberg',
 'adammckay',
 'adammilosmalley',
 'adams',
 'adamsandler',
 'adamscott',
 'adamshankman',
 'adaptation',
 'addiction',
 'adewaleakinnuoye',
 'adopted',
 'adoption',
 'adrienbrody',
 'adult',
 'adultery',
 'adventure',
 'adventures',
 'advice',
 'affair',
 'afghanistan',
 'africa',
 'african',
 'aftercreditsstinger',
 'afterlife',
 'agbaje',
 'age',
 'aged',
 'agency',
 'agent',
 'agents',
 'aging',
 'ago',
 'agree',
 'agrees',
 'aguil

In [43]:
# build the model
from sklearn.metrics.pairwise import cosine_similarity
similarity =cosine_similarity(vector)

In [44]:
similarity.shape
# is change the field is decrease karena cut di gradient learning nya
# the result is like that for oene sample
similarity[0]
# similarity will be one

array([1.        , 0.06775964, 0.02290694, ..., 0.01630387, 0.01749546,
       0.        ])

In [45]:
#function of recomended system
def recomend(movie):
    index = new_movies[new_movies["title"]==movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key=lambda x : x[1])
    for i in distances[1:6]:# get 6 value for distances recomendasi 
        print(new_movies.iloc[i[0]].title)


In [46]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x : x[1])

[(0, 1.0),
 (25, 0.2788730589952011),
 (2409, 0.2673935429164744),
 (282, 0.24174688920761414),
 (279, 0.2330745053742094),
 (3443, 0.18802535827258876),
 (539, 0.18664987997671081),
 (587, 0.17727200782610344),
 (1216, 0.15731330124142315),
 (1444, 0.15569978883230456),
 (2786, 0.15194743527951726),
 (3730, 0.15075567228888181),
 (4192, 0.1462544848254261),
 (1920, 0.1448413648755803),
 (1109, 0.1333662673423161),
 (4048, 0.13196793041081598),
 (507, 0.13081398803354571),
 (199, 0.13065491598369758),
 (3327, 0.12974982402692048),
 (83, 0.12792042981336627),
 (108, 0.12712834523274566),
 (168, 0.1266600992762247),
 (94, 0.12208664567220492),
 (28, 0.12062447740929097),
 (2971, 0.11927604449408552),
 (3538, 0.11891767800211261),
 (3608, 0.11844484241978358),
 (1948, 0.11826247919781654),
 (1537, 0.1172544117802414),
 (184, 0.11725441178024137),
 (17, 0.11653431646335018),
 (4405, 0.11605177063713192),
 (74, 0.11581371547232745),
 (942, 0.11565903932254226),
 (973, 0.11565903932254226),


In [47]:
# this is testing result
recomend("Gandhi")

The Wind That Shakes the Barley
A Passage to India
Ramanujan
Guiana 1838
Lion of the Desert


In [48]:
# save model used pickle
import pickle
#save data for model
pickle.dump(new_movies,open("movie_list.pkl","wb"))#weight binary format model saved
#save model similarity yang di train dnegan data new movie
pickle.dump(similarity,open("similarity.pkl","wb"))